In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 91.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 102.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 26.2 MB/s eta 0:00:00


In [5]:
tokenizer_name = "ainize/klue-bert-base-mrc"
model_name = "ainize/klue-bert-base-mrc"

In [6]:
import json
import random

import torch
import numpy as np
from tqdm.notebook import tqdm
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, AdamW

In [7]:
with open("/content/drive/MyDrive/WARNING_PRIVATE_FOLDER/goorm_nlp_8th_group3/goorm_nlp_8th_group3/project2/train.json", 'rb') as f:
    input_dict = json.load(f)
input_dict["data"][0]

{'title': '제주도 장마 시작 … 중부는 이달 말부터',
 'paragraphs': [{'context': '올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중부지방은 예년보다 사나흘 정도 늦은 이달 말께 장마가 시작될 전망이다.17일 기상청에 따르면 제주도 남쪽 먼바다에 있는 장마전선의 영향으로 이날 제주도 산간 및 내륙지역에 호우주의보가 내려지면서 곳곳에 100㎜에 육박하는 많은 비가 내렸다. 제주의 장마는 평년보다 2~3일, 지난해보다는 하루 일찍 시작됐다. 장마는 고온다습한 북태평양 기단과 한랭 습윤한 오호츠크해 기단이 만나 형성되는 장마전선에서 내리는 비를 뜻한다.장마전선은 18일 제주도 먼 남쪽 해상으로 내려갔다가 20일께 다시 북상해 전남 남해안까지 영향을 줄 것으로 보인다. 이에 따라 20~21일 남부지방에도 예년보다 사흘 정도 장마가 일찍 찾아올 전망이다. 그러나 장마전선을 밀어올리는 북태평양 고기압 세력이 약해 서울 등 중부지방은 평년보다 사나흘가량 늦은 이달 말부터 장마가 시작될 것이라는 게 기상청의 설명이다. 장마전선은 이후 한 달가량 한반도 중남부를 오르내리며 곳곳에 비를 뿌릴 전망이다. 최근 30년간 평균치에 따르면 중부지방의 장마 시작일은 6월24~25일이었으며 장마기간은 32일, 강수일수는 17.2일이었다.기상청은 올해 장마기간의 평균 강수량이 350~400㎜로 평년과 비슷하거나 적을 것으로 내다봤다. 브라질 월드컵 한국과 러시아의 경기가 열리는 18일 오전 서울은 대체로 구름이 많이 끼지만 비는 오지 않을 것으로 예상돼 거리 응원에는 지장이 없을 전망이다.',
   'qas': [{'question': '북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?',
     'answers': [{'text': '한 달가량', 'answer_start': 478},
      {'text': '한 달', 'answer_start': 478}],
     'guid': '798db07f0b9046759deed9d4a

In [8]:
input_dict["data"][1]

{'title': '부산정보산업진흥원, 과기부 지역SW서비스사업화 지원사업 4개 과제 선정',
 'paragraphs': [{'context': '부산시와 (재)부산정보산업진흥원(원장 이인숙)이 ‘2020~2021년 지역SW서비스사업화 지원사업’ 공모사업에 4개 과제가 선정되어 본격적인 사업 착수에 나선다. 과학기술정보통신부가 주관하는 ‘지역SW서비스사업화 지원사업’은 강소SW기업 및 초기 스타트업의 SW서비스 사업화 지원과 신시장 진출 지원을 통해 기업 경쟁력 강화와 지역경제 활성화를 도모하는 사업이다. 올해부터 2개년으로 진행되며, 국비와 시비, 민자 등 2년간 약 37억원의 예산이 투입된다. 앞서 진흥원은 부산의 미래 먹거리산업인 스마트해양, 지능형기계, 지능정보서비스 분야로 사전 수요조사를 진행했고, 평가를 통해 선정된 5개 과제를 공모사업에 신청했다. 그 결과 부산의 4개 과제가 최종 선정되는 쾌거를 거뒀다. 당 사업은 전국 진흥기관을 대상으로 공모를 시작해, 총 17개 지역에서 42개 과제가 선정되었으며, 4개 과제가 선정된 곳은 부산과 강원지역 뿐이다. 금번 선정된 과제들은 ‘인공지능융합센서와 서보 이송 로봇을 이용한 전단보강재의 자동용접시스템 개발’ 등 총 4개 과제다. 부산시가 지원하고, 부산정보산업진흥원과 지역기업, 대학, 연구소 등이 컨소시엄을 구성하여 기술개발 및 사업화 지원을 추진한다. 2개의 Track으로 구분되는 이번사업은 Track 1(SW중소기업)에서 ㈜에이아이플랫폼, 엔컴(주), Track 2(스타트업)에서는 ㈜토즈, 삼보테크놀로지를 지원한다. ○ ‘Track 1‘의 (주)에이아이플랫폼이 주관기업으로 진행하는 <인공지능 기반 망막 내 아밀로이드 플라크 영상 분석을 통한 치매조기진단 플랫폼 상용화>는 치매 확진의 원인이 되는 중요 단백질(아밀로이드 플라크)을 자체개발 관측장비로 진단한다. 이를 통해 치매를 조기 발견하여, 각종 경제적 비용과 치료 및 예방 등 사회적 문제를 해 결하고 시민들이 쉽게 접근 가능한 실효성 있는 치매관리

In [9]:
from copy import deepcopy
def split_input_dict(input_dict, ratio = 0.1, seed = 42):
    split_point = int(len(input_dict['data']) * ratio)
    random.seed(seed)
    random.shuffle(input_dict['data'])
    valid_dict = deepcopy(input_dict)
    train_dict = input_dict

    valid_dict['data'] = input_dict['data'][:split_point]
    train_dict['data'] = input_dict['data'][split_point:]
    return train_dict, valid_dict

In [10]:
def read_input(path):
    with open(path, 'rb') as f:
        input_dict = json.load(f)
    train_dict,valid_dict =split_input_dict(input_dict)
    train_contexts = []
    train_questions = []
    train_answers = []
    for group in tqdm(train_dict['data']):       #딕셔너리 하나씩 꺼낸다.
        for passage in group['paragraphs']:     #딕셔너리의 paragraphs
            context = passage['context']        #paragraphs의 context
            for qa in passage['qas']:           #paragraphs의 qas
                question = qa['question']       #paragraphs의 question
                for answer in qa['answers']:    #question의 answers
                    train_contexts.append(context)    #answers의 한 answer당 해당하는 context 저장
                    train_questions.append(question)  #answers의 한 answer당 해당하는 question 저장
                    train_answers.append(answer)      #answers의 한 answer 저장
  
    valid_contexts = []
    valid_questions = []
    valid_answers = []
    for group in tqdm(valid_dict['data']):       #딕셔너리 하나씩 꺼낸다.
        for passage in group['paragraphs']:     #딕셔너리의 paragraphs
            context = passage['context']        #paragraphs의 context
            for qa in passage['qas']:           #paragraphs의 qas
                question = qa['question']       #paragraphs의 question
                for answer in qa['answers']:    #question의 answers
                    valid_contexts.append(context)    #answers의 한 answer당 해당하는 context 저장
                    valid_questions.append(question)  #answers의 한 answer당 해당하는 question 저장
                    valid_answers.append(answer)      #answers의 한 answer 저장
    return train_contexts, train_questions, train_answers, valid_contexts, valid_questions, valid_answers

In [11]:
def add_end_idx(answers, contexts):
    for answer, context in zip(answers, contexts):
        gold_text = answer['text']
        start_idx = answer['answer_start']
        end_idx = start_idx + len(gold_text)

        if context[start_idx:end_idx] == gold_text:
            answer['answer_end'] = end_idx
        elif context[start_idx-1:end_idx-1] == gold_text:
            print("there is an unitended error in dataset") #이렇게까지 할 필요가 있나?
            answer['answer_start'] = start_idx - 1
            answer['answer_end'] = end_idx - 1
        elif context[start_idx-2:end_idx-2] == gold_text:
            print("there is an unitended error in dataset")
            answer['answer_start'] = start_idx - 2
            answer['answer_end'] = end_idx - 2

In [12]:
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)

Downloading:   0%|          | 0.00/499 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/609 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/248k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/495k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [13]:
question = "북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?"
context = "올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중부지방은 예년보다 사나흘 정도 늦은 이달 말께 장마가 시작될 전망이다.17일 기상청에 따르면 제주도 남쪽 먼바다에 있는 장마전선의 영향으로 이날 제주도 산간 및 내륙지역에 호우주의보가 내려지면서 곳곳에 100㎜에 육박하는 많은 비가 내렸다. 제주의 장마는 평년보다 2~3일, 지난해보다는 하루 일찍 시작됐다. 장마는 고온다습한 북태평양 기단과 한랭 습윤한 오호츠크해 기단이 만나 형성되는 장마전선에서 내리는 비를 뜻한다.장마전선은 18일 제주도 먼 남쪽 해상으로 내려갔다가 20일께 다시 북상해 전남 남해안까지 영향을 줄 것으로 보인다. 이에 따라 20~21일 남부지방에도 예년보다 사흘 정도 장마가 일찍 찾아올 전망이다. 그러나 장마전선을 밀어올리는 북태평양 고기압 세력이 약해 서울 등 중부지방은 평년보다 사나흘가량 늦은 이달 말부터 장마가 시작될 것이라는 게 기상청의 설명이다. 장마전선은 이후 한 달가량 한반도 중남부를 오르내리며 곳곳에 비를 뿌릴 전망이다. 최근 30년간 평균치에 따르면 중부지방의 장마 시작일은 6월24~25일이었으며 장마기간은 32일, 강수일수는 17.2일이었다.기상청은 올해 장마기간의 평균 강수량이 350~400㎜로 평년과 비슷하거나 적을 것으로 내다봤다. 브라질 월드컵 한국과 러시아의 경기가 열리는 18일 오전 서울은 대체로 구름이 많이 끼지만 비는 오지 않을 것으로 예상돼 거리 응원에는 지장이 없을 전망이다."
tokenizer(context, question)

{'input_ids': [2, 1446, 22555, 11477, 2116, 3932, 2210, 6530, 27135, 3670, 2367, 2062, 18, 3671, 886, 9775, 16311, 2073, 12982, 2178, 2062, 15513, 3309, 3681, 798, 2073, 5277, 1041, 2678, 11477, 2116, 3670, 2651, 4016, 28674, 18, 3932, 2210, 11945, 2170, 3881, 2460, 6530, 7831, 1060, 10526, 2170, 1513, 2259, 11477, 2165, 2020, 2079, 3979, 6233, 3814, 6530, 24028, 1116, 12468, 17552, 2170, 24902, 3802, 2178, 2116, 23772, 31369, 5844, 2170, 3911, 3569, 2170, 10760, 2205, 2259, 1039, 2073, 1187, 2116, 5740, 2062, 18, 4364, 2079, 11477, 2259, 18673, 2178, 2062, 22, 97, 23, 2210, 16, 3736, 2178, 4000, 4051, 5947, 3670, 2367, 2062, 18, 11477, 2259, 19880, 2062, 2219, 2470, 1174, 18956, 26797, 2145, 1891, 2398, 1322, 2399, 2470, 22152, 2128, 2292, 2097, 26797, 2052, 4026, 4605, 2496, 2259, 11477, 2165, 2020, 27135, 4848, 2259, 1187, 2138, 936, 4538, 18, 11477, 2165, 2020, 2073, 3801, 2210, 6530, 1060, 7831, 7755, 6233, 12314, 4795, 3619, 2210, 2678, 3690, 25848, 2097, 4997, 18787, 2299, 2118,

In [14]:
class KlueDataset(Dataset):
    def __init__(self, contexts, questions, answers, model_max_position_embedings, tokenizer):
        self.tokenizer = tokenizer
        self.answers = answers
        self.questions = questions
        self.contexts = contexts
        self.model_max_position_embedings = model_max_position_embedings
        print("Tokenizing ...")
        self.encodings = self.tokenizer(self.contexts, 
                                        self.questions,
                                        max_length=512, #512 truncation
                                        truncation=True,
                                        padding="max_length",
                                        return_token_type_ids=False)
        print("Done !!!")
        self.add_token_positions()
        
    def add_token_positions(self):
        start_positions = []
        end_positions = []
        for i in range(len(self.answers)):
            start_positions.append(self.encodings.char_to_token(i, self.answers[i]['answer_start']))
            end_positions.append(self.encodings.char_to_token(i, self.answers[i]['answer_end'] - 1)) # -1으로 : 진짜로 답이 있는 end_position 의 인덱스를 구함.(char_to_token은 인덱스를 구함)
            #https://huggingface.co/docs/tokenizers/v0.13.2/en/api/encoding#tokenizers.Encoding.char_to_token

            # positions 값이 None 값이라면, answer가 포함된 context가 잘렸다는 의미
            if start_positions[-1] is None:
                print("there is an error 1")
                start_positions[-1] = self.model_max_position_embedings
            if end_positions[-1] is None:
                print("there is an error 2")
                end_positions[-1] = self.model_max_position_embedings

        self.encodings.update({'start_positions': start_positions, 'end_positions': end_positions})

        
    def get_data(self):
        return {"contexts":self.contexts, 'questions':self.questions, 'answers':self.answers}
    
    
    def get_encodings(self):
        return self.encodings
        
    
    def __getitem__(self, idx):
        return {key:torch.tensor(val[idx]) for key, val in self.encodings.items()}
    
    def __len__(self):
        return len(self.encodings['input_ids'])

In [ ]:
train_contexts, train_questions, train_answers, valid_contexts, valid_questions, valid_answers = read_input("/content/drive/MyDrive/WARNING_PRIVATE_FOLDER/goorm_nlp_8th_group3/goorm_nlp_8th_group3/project2/train.json")
add_end_idx(train_answers, train_contexts)                                                      #anwer 마다 answer_end 달아준다.
train_dataset = KlueDataset(train_contexts, train_questions, train_answers, 512, tokenizer)

add_end_idx(valid_answers, valid_contexts)                                                      #anwer 마다 answer_end 달아준다.
valid_dataset = KlueDataset(valid_contexts, valid_questions, valid_answers, 512, tokenizer)

In [16]:
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [17]:
EPOCH = 3
LEARNING_RATE = 5e-5
BATCH_SIZE = 8

In [18]:
def train_runner(model, train_dataset, valid_dataset , batch_size, num_train_epochs, learning_rate):
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    
    model.to(device)
    model.train()
    train_dataloader = DataLoader(dataset=train_dataset, batch_size=batch_size)
    valid_dataloader = DataLoader(dataset = valid_dataset, batch_size = batch_size)

    lowest_total_valid_loss = 9999.

    global_total_step = len(train_dataloader) * num_train_epochs
    optimizer = AdamW(model.parameters(), lr=learning_rate, weight_decay=0)
    print("TRAIN START")
    with tqdm(total=global_total_step, unit='step') as t:
        total = 0
        total_loss = 0
        for epoch in range(num_train_epochs):
            for iteration,batch in enumerate(train_dataloader):
                optimizer.zero_grad()
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                start_positions = batch['start_positions'].to(device)
                end_positions = batch['end_positions'].to(device)
                outputs = model(input_ids,
                             attention_mask=attention_mask,
                             start_positions=start_positions,
                             end_positions=end_positions)
                loss = outputs.loss
                loss.backward()
                optimizer.step()
                
                batch_loss = loss.item() * len(input_ids)
                total += len(input_ids)
                total_loss += batch_loss
                global_total_step += 1
                t.set_postfix(loss="{:.6f}".format(total_loss / total), batch_loss="{:.6f}".format(batch_loss))
                t.update(1)
                
                del input_ids
                del attention_mask
                del start_positions
                del end_positions
                del outputs
                del loss

                ## validation ##
                if iteration != 0 and iteration % int(len(train_dataloader) / 5) == 0:
                    total_valid_loss = 0
                    for batch_val in valid_dataloader:
                        model.eval()
                        optimizer.zero_grad()

                        input_ids = batch_val['input_ids'].to(device)
                        attention_mask = batch_val['attention_mask'].to(device)
                        start_positions = batch_val['start_positions'].to(device)
                        end_positions = batch_val['end_positions'].to(device)
                
                        with torch.no_grad():
                            outputs = model(input_ids,
                                    attention_mask=attention_mask,
                                    start_positions=start_positions,
                                    end_positions=end_positions)
                            loss = outputs.loss
                            total_valid_loss += loss.item()
                    
                    if total_valid_loss < lowest_total_valid_loss:
                        print(f"lowest_total_valid_loss: {total_valid_loss} epoch : {epoch} iteration : {iteration}")
                        torch.save(model.state_dict(),'./output_model_best')
                        lowest_total_valid_loss = total_valid_loss
                ## validation ##

    #model.save_pretrained("./klue_output_model")
    print("TRAIN END")

In [19]:
train_runner(model,train_dataset,valid_dataset, BATCH_SIZE, EPOCH, LEARNING_RATE)

TRAIN START


/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/5958 [00:00<?, ?step/s]

lowest_total_valid_loss: 161.01216702908278 epoch : 0 iteration : 397
TRAIN END


In [20]:
def read_dev_klue(path):
    with open(path, 'rb') as f:
        klue_dict = json.load(f)

    contexts = []
    questions = []
    answers = []
    guids = []

    for group in tqdm(klue_dict['data']):
        for passage in group['paragraphs']:
            context = passage['context']
            for qa in passage['qas']:
                question = qa['question']
                guid = qa['guid']
                #temp_answer = []
                #for answer in qa['answers']:
                    #temp_answer.append(answer['text'])
                #if len(temp_answer) != 0: # answers의 길이가 0 == 답변할 수 없는 질문
                    #contexts.append(context)
                    #questions.append(question)
                    #answers.append(temp_answer)
                contexts.append(context)##
                questions.append(question)##
                guids.append(guid)

    #return contexts, questions, answers
    return contexts, questions , guids

In [21]:
#dev_contexts, dev_questions, dev_answers = read_dev_klue("/content/drive/MyDrive/WARNING_PRIVATE_FOLDER/goorm_nlp_8th_group3/goorm_nlp_8th_group3/project2/test.json")
dev_contexts, dev_questions, dev_guids = read_dev_klue("/content/drive/MyDrive/WARNING_PRIVATE_FOLDER/goorm_nlp_8th_group3/goorm_nlp_8th_group3/project2/test.json")

  0%|          | 0/3709 [00:00<?, ?it/s]

In [31]:


def prediction(contexts, questions, guids):
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

    model.load_state_dict(torch.load('./output_model_best'))
    model.to(device)
    
    model.eval()
    
    result = []
    
    with torch.no_grad():
        
        for context, question, guid in zip(contexts, questions, guids):
            encodings = tokenizer(context, question, max_length=512, truncation=True,
                                     padding="max_length", return_token_type_ids=False)
            encodings = {key: torch.tensor([val]) for key, val in encodings.items()}
            
            input_ids = encodings["input_ids"].to(device)
            attention_mask = encodings["attention_mask"].to(device)
            
            outputs = model(input_ids, attention_mask=attention_mask)
            start_logits, end_logits = outputs.start_logits, outputs.end_logits
            token_start_index, token_end_index = start_logits.argmax(dim=-1), end_logits.argmax(dim=-1)
            pred_ids = input_ids[0][token_start_index: token_end_index + 1]
            if token_start_index > token_end_index:
                print("이상값 : ",tokenizer.decode(pred_ids))
                pred = tokenizer.decode(pred_ids)
            else:
                pred = tokenizer.decode(pred_ids)
                #pred = pred[:10] 
            

            tp = (guid,pred)
            
            result.append(tp)
    return result

In [32]:
pred_answers = prediction(dev_contexts, dev_questions, dev_guids)
pred_answers

이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  
이상값 :  


[('d14cb73158624cf094c546d856fd3c80', '뉴 740Li 25주년 에디션'),
 ('906631384e91493ebe1c7f34aea6f241', ''),
 ('35e61dcb479643448a2cb7d326ae50a6', '링크트인과 페이스북'),
 ('075e761b370040cb9041eecd39afc27c', '링크트인과 페이스북'),
 ('e67ed38f3dd944be94d5b4c53731f334', '마드리드'),
 ('80cacfdfe76442b0b39568e8b193d066', '국제 원자재값'),
 ('78a80cca941c46379a5c65b2f1392f7c', '5조원'),
 ('5c19b9781f8a4f0faa9045274c38a8b4', '‘ 운영허가 인증서 ’'),
 ('d2844b7141cb4a2594d433e9c1e5cd0c', '성형'),
 ('be39b91f52a04b0aaafc96ff505cb2fc', '노르웨이'),
 ('98fe72c173f642e29f9284f1769fd4f6', '16종'),
 ('1756f8a643124b52aa773a0f38b7b6fa', '필리핀'),
 ('1f1ef695105049079e093bae6f4ad8e7', '김교성'),
 ('acbc4be5f9d04b76a30451f8c6baf277', '1118. 3원'),
 ('393657048abd435b9059b00ed54dcc47', '존 위클리프'),
 ('08ee8dae9f6744cf91d79e0782426d33', '존 위클리프'),
 ('0fd8aca8b9844e3d85e0379272600e3b', '신성 로마 제국의 황제 지기스문트'),
 ('fa773f05fd224cbab0327d1459e38f8a', '복정역'),
 ('89810801a2f446ef9ea0b74cdc478f3c', '내일'),
 ('30a8f6b62fa142209b241d35e308002b', '경희대 음대에서 교편을 잡은 뒤 서울대'),

In [33]:
import csv
f = open('ainize_klue-bert-base-mrc.csv','w', newline='')
wr = csv.writer(f)
wr.writerow(['Id','Predicted'])

for tp in pred_answers:
    wr.writerow([tp[0],tp[1]])

f.close()